### Libraries import

In [1]:
import pandas as pd # Dataframes management
from zipfile import ZipFile  # Files compressed management
import os # Files management along OS
import re

In [2]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

### Data import

In [3]:
# Specifying the name of the zip file
file = "Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.zip"
  
# Open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # List all the contents of the zip file
    zip.printdir() 
  
    # Extract all files
    print('extraction...') 
    zip.extractall("Inputs/Modificados - Atmira_Pharma_Visualization") 
    print('Done!')

#Import CSV to pandas
itemsOrdered = pd.read_csv("Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.csv")
print("CSV imported to Pandas successfully")

# Remove uncompressed CSV file
os.remove("Inputs/Modificados - Atmira_Pharma_Visualization/items_ordered_2years_V2.csv")
print("Original CSV removed to preserve repo health")

File Name                                             Modified             Size
items_ordered_2years_V2.csv                    2022-02-23 01:28:04    150372498
extraction...
Done!
CSV imported to Pandas successfully
Original CSV removed to preserve repo health


In [4]:
itemsOrdered.head(3)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,562ba4ba9aaf2a1e926842deca19271d,2bca87c2ab10b07feef290a9be47f316,2017-01-01 00:20:33,6409.0,1.0,13.4204,17.17,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
1,562ba4ba9aaf2a1e926842deca19271d,53e12ac4b579f0c4b1c4f77d50bf8792,2017-01-01 00:20:33,5133.0,1.0,18.5351,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
2,562ba4ba9aaf2a1e926842deca19271d,d19d6df5cd6a1e741408b70aa2767bf4,2017-01-01 00:20:33,5125.0,1.0,18.5260,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019


In [5]:
del zip

### Arreglos para facilitar el webscrapping

In [6]:
itemsOrdered["city"].replace(to_replace={'San Vicente del Raspeig':'Sant Vicent del Raspeig'}, inplace=True)

In [7]:
itemsOrdered.loc[itemsOrdered['zipcode'].eq('30139') & itemsOrdered['city'].eq('Murcia'), "city"] = "EL RAAL"

In [8]:
itemsOrdered.loc[itemsOrdered['zipcode'].eq('29039')] = "28039"

### USO DE WEBSCRAPPING PARA CÓDIGO POSTAL

In [9]:
masterScrapping = []

In [10]:
rawDataZipcode = list(zip(itemsOrdered["city"].tolist(), itemsOrdered["zipcode"].tolist()))
rawDataZipcode = rawDataZipcode[:int(len(rawDataZipcode)/30)]

In [11]:
def AcentosLimpiador(text):
	acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
	for ele in acentos:
		if ele in text:
			text = text.replace(ele, acentos[ele])
	return text

In [14]:
def zipCodeManipulation(city, zipcode, query, saved = False):
  
    if saved == False:
        listTestingZipcode = saveGeoInfo.split("\n")
        indexMatchRegex = list(map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(r"[a-z][A-Z0-9]", x)], listTestingZipcode))

        resultScrapClean = []
        for pas,resultScrap in enumerate(listTestingZipcode):
            for pos,ele in enumerate(indexMatchRegex[pas]):
                if len(indexMatchRegex[pas])==2:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                        resultScrapClean.append(txt)
                elif len(indexMatchRegex[pas])==3:
                    if pos ==0:
                        txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
                    elif pos ==1:
                        txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                    elif pos ==2:
                        txt = txt[:ele[0]+3]+","+txt[ele[1]+1:]
                        resultScrapClean.append(txt)

        resultScrapListed = []
        for element in resultScrapClean:
            resultScrapListed.append(element.split(","))

        resultScrapRearr = [(element[0], element[1], element[-2], element[-1]) for element in resultScrapListed]
        
        resultZip = []
        for element in resultScrapRearr:
            if element[2].lower() == AcentosLimpiador(city.lower()):
                resultZip.append(element)
                break


    return resultZip[0]

In [27]:
try:
    opts = Options()
    opts.binary_location = "/usr/bin/chromium-browser"
    driver = webdriver.Chrome(options=opts)

    driver.get("https://worldpostalcode.com/lookup")
    driver.set_page_load_timeout(5)

    for element in rawDataZipcode:
        insertZipcode =driver.find_element(By.ID,"search")
        insertZipcode.clear()
        insertZipcode.send_keys(element[1])
        clickButtonZipcode =driver.find_element(By.CLASS_NAME,"submit")
        clickButtonZipcode.click()
        driver.set_page_load_timeout(5)

        getGeoInfo = driver.find_element(By.CLASS_NAME,"search_units")
        saveGeoInfo = getGeoInfo.text

        if element[0] not in masterScrapping:
            zipCodeDef = zipCodeManipulation(element[0], element[1], saveGeoInfo)
        else:
            zipCodeDef = zipCodeManipulation(element[0], element[1], saveGeoInfo, saved = True)

        masterScrapping.append(zipCodeDef)

    driver.close()

except IndexError:
    driver.close()
    raise Exception(f"El fallo se ha producido con ciudad: {element[0]}, zipcode: {element[1]}")

SessionNotCreatedException: Message: session not created
from disconnected: received Inspector.detached event
  (Session info: chrome=96.0.4664.110)
Stacktrace:
#0 0x5627fc40aee3 <unknown>
#1 0x5627fbed8608 <unknown>
#2 0x5627fbec4fa5 <unknown>
#3 0x5627fbec4a05 <unknown>
#4 0x5627fbec3f35 <unknown>
#5 0x5627fbec2d43 <unknown>
#6 0x5627fbec3093 <unknown>
#7 0x5627fbec26c1 <unknown>
#8 0x5627fbed1d30 <unknown>
#9 0x5627fbec2663 <unknown>
#10 0x5627fbec39d9 <unknown>
#11 0x5627fbec2d43 <unknown>
#12 0x5627fbec3093 <unknown>
#13 0x5627fbec26c1 <unknown>
#14 0x5627fbecf59d <unknown>
#15 0x5627fbec2663 <unknown>
#16 0x5627fbec39d9 <unknown>
#17 0x5627fbec2d43 <unknown>
#18 0x5627fbec3093 <unknown>
#19 0x5627fbec26c1 <unknown>
#20 0x5627fbecaecb <unknown>
#21 0x5627fbec2663 <unknown>
#22 0x5627fbec39d9 <unknown>
#23 0x5627fbec2d43 <unknown>
#24 0x5627fbec3093 <unknown>
#25 0x5627fbec26c1 <unknown>
#26 0x5627fbec9efa <unknown>
#27 0x5627fbec2663 <unknown>
#28 0x5627fbec39d9 <unknown>
#29 0x56

In [17]:
len(masterScrapping)

9

In [30]:
asd = pd.DataFrame(masterScrapping, columns=["Country","Region","City","Zipcode"])

In [32]:
asd.head(50)

,Country,Region,City,Zipcode
0,Spain,Comunidad Valenciana,Valencia,46019
1,Spain,Comunidad Valenciana,Valencia,46019
2,Spain,Comunidad Valenciana,Valencia,46019
3,Spain,Andalucia,Almeria,04009
4,Spain,Andalucia,Almeria,04009
5,Spain,Comunidad Valenciana,Valencia,46019
6,Spain,Comunidad Valenciana,Valencia,46019
7,Spain,Comunidad Valenciana,Valencia,46019
8,Spain,Andalucia,Almeria,04009
9,Spain,Comunidad Valenciana,Valencia,46019


In [46]:
qwerty = '"Huesos, articulaciones y músculos, Colágeno'

In [47]:
asd =  [element for element in [rowex for rowex in qwerty] if '"' in element]

In [48]:
asd

['"']

In [25]:
import re

In [55]:
test2 = re.findall("([^;]*)", element)

In [57]:
test2

['Colágeno', '']

In [ ]:
mask = categories_sep[['cat2', 'cat3', '2']].isna().all(axis=1)

In [ ]:
categories_sep[mask, ['cat2', 'cat3', '2']] = categories_sep[mask, ['cat2', 'cat3', '2']].shift(1, axis=1)

In [ ]:
categories_sep_2_3.iloc[lambda x: x == None]